In [9]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'  # 设置Hugging Face镜像地址

import dashscope
from dashscope_api import api
dashscope.api_key = api

In [23]:
import requests

# 视频的bvid
bvid = "BV1Un9vY9EX7"

# 获取视频信息的API（用于获取cid）
info_url = f"https://api.bilibili.com/x/web-interface/view?bvid={bvid}"

# 请求头，模拟浏览器请求
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Referer": "https://www.bilibili.com"
}

# 获取视频信息
response = requests.get(info_url, headers=headers)
data = response.json()

if data["code"] == 0:
    # 获取cid
    cid = data["data"]["cid"]
    print("获取到cid:", cid)

    # 获取音频流地址的API
    play_url = f"https://api.bilibili.com/x/player/playurl?bvid={bvid}&cid={cid}&fnval=16"

    # 发送请求
    play_response = requests.get(play_url, headers=headers)
    play_data = play_response.json()

    if play_data["code"] == 0:
        # 解析音频流地址
        audio_url = play_data["data"]["dash"]["audio"][0]["base_url"]
        print("音频流地址:", audio_url)

        # 下载音频
        audio_response = requests.get(audio_url, headers=headers)
        with open("audio.m4s", "wb") as f:
            f.write(audio_response.content)
        print("音频下载完成，保存为 audio.m4s")
    else:
        print("获取音频流失败:", play_data["message"])
else:
    print("获取视频信息失败:", data["message"])

获取到cid: 28664660478
音频流地址: https://xy120x198x101x74xy.mcdn.bilivideo.cn:8082/v1/resource/28664660478-1-30280.m4s?agrr=0&build=0&buvid=&bvc=vod&bw=23728&deadline=1741013240&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&f=u_0_0&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&og=cos&oi=794575799&orderid=0%2C3&os=coso1bv&platform=pc&sign=769c54&traceid=trthOBjzPrNBHT_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform%2Cog&upsig=e623d150f882e4f2743f94303592bfea
音频下载完成，保存为 audio.m4s


In [24]:
audio_urls = play_data["data"]["dash"]["audio"]
audio_url = audio_urls[0]["baseUrl"]

audio_url

'https://xy120x198x101x74xy.mcdn.bilivideo.cn:8082/v1/resource/28664660478-1-30280.m4s?agrr=0&build=0&buvid=&bvc=vod&bw=23728&deadline=1741013240&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&f=u_0_0&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&og=cos&oi=794575799&orderid=0%2C3&os=coso1bv&platform=pc&sign=769c54&traceid=trthOBjzPrNBHT_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform%2Cog&upsig=e623d150f882e4f2743f94303592bfea'

In [25]:
from http import HTTPStatus
from dashscope.audio.asr import Transcription
import json


transcribe_response = Transcription.async_call(
    model='paraformer-v2',
    file_urls=[audio_url],
    language_hints=['zh', 'en']  # “language_hints”只支持paraformer-v2和paraformer-realtime-v2模型
)

while True:
    if transcribe_response.output.task_status == 'SUCCEEDED' or transcribe_response.output.task_status == 'FAILED':
        break
    transcribe_response = Transcription.fetch(task=transcribe_response.output.task_id)

if transcribe_response.status_code == HTTPStatus.OK:
    print(json.dumps(transcribe_response.output, indent=4, ensure_ascii=False))
    print('transcription done!')

{
    "task_id": "969213a4-490b-43a7-91dc-bffd100b3f39",
    "task_status": "SUCCEEDED",
    "submit_time": "2025-03-03 20:47:26.668",
    "scheduled_time": "2025-03-03 20:47:26.711",
    "end_time": "2025-03-03 20:47:45.462",
    "results": [
        {
            "file_url": "https://xy120x198x101x74xy.mcdn.bilivideo.cn:8082/v1/resource/28664660478-1-30280.m4s?agrr=0&build=0&buvid=&bvc=vod&bw=23728&deadline=1741013240&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&f=u_0_0&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&og=cos&oi=794575799&orderid=0%2C3&os=coso1bv&platform=pc&sign=769c54&traceid=trthOBjzPrNBHT_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform%2Cog&upsig=e623d150f882e4f2743f94303592bfea",
            "transcription_url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/prod/p

In [26]:
transcription_url = transcribe_response.output.results[0]['transcription_url']
transcription_url


'https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/prod/paraformer-v2/20250303/20%3A47/e14caa6e-10e0-40d9-9756-5345c1882d7f-1.json?Expires=1741092465&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=CJIM1QJcl7FoWrFkeiHaihJlujg%3D'

In [27]:
# 下载识别结果 JSON 文件
response = requests.get(transcription_url)
if response.status_code == 200:
    transcription_result = response.json()
    


In [28]:
transcription_result

{'file_url': 'https://xy120x198x101x74xy.mcdn.bilivideo.cn:8082/v1/resource/28664660478-1-30280.m4s?agrr=0&build=0&buvid=&bvc=vod&bw=23728&deadline=1741013240&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&f=u_0_0&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&og=cos&oi=794575799&orderid=0%2C3&os=coso1bv&platform=pc&sign=769c54&traceid=trthOBjzPrNBHT_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform%2Cog&upsig=e623d150f882e4f2743f94303592bfea',
 'properties': {'audio_format': 'aac',
  'channels': [0, 1],
  'original_sampling_rate': 48000,
  'original_duration_in_milliseconds': 819242},
 'transcripts': [{'channel_id': 0,
   'content_duration_in_milliseconds': 706140,
   'text': '不知从什么时候开始，各家手机厂商似乎都在不约而同的做一件事情，就是把你的手机和相机的边界变得非常的模糊。很多时候你会怀疑我花钱买的这个东西到底是一台更像手机的相机，还是一台更像相机的手机。整个问题可能没有答案，但是你会看到各家厂商有各种魔改的方案

In [29]:
# 查看转录结果的结构
from pprint import pprint as pp
pp(transcription_result, depth=2)

{'file_url': 'https://xy120x198x101x74xy.mcdn.bilivideo.cn:8082/v1/resource/28664660478-1-30280.m4s?agrr=0&build=0&buvid=&bvc=vod&bw=23728&deadline=1741013240&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&f=u_0_0&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&og=cos&oi=794575799&orderid=0%2C3&os=coso1bv&platform=pc&sign=769c54&traceid=trthOBjzPrNBHT_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform%2Cog&upsig=e623d150f882e4f2743f94303592bfea',
 'properties': {'audio_format': 'aac',
                'channels': [...],
                'original_duration_in_milliseconds': 819242,
                'original_sampling_rate': 48000},
 'transcripts': [{...}]}


In [30]:
pp(transcription_result['transcripts'])

[{'channel_id': 0,
  'content_duration_in_milliseconds': 706140,
  'sentences': [{'begin_time': 2060,
                 'end_time': 10352,
                 'sentence_id': 1,
                 'text': '不知从什么时候开始，各家手机厂商似乎都在不约而同的做一件事情，就是把你的手机和相机的边界变得非常的模糊。',
                 'words': [{'begin_time': 2060,
                            'end_time': 2229,
                            'punctuation': '',
                            'text': '不'},
                           {'begin_time': 2229,
                            'end_time': 2398,
                            'punctuation': '',
                            'text': '知'},
                           {'begin_time': 2398,
                            'end_time': 2567,
                            'punctuation': '',
                            'text': '从'},
                           {'begin_time': 2567,
                            'end_time': 2736,
                            'punctuation': '',
                            'text': '什'},
              

In [31]:
transcription_result['transcripts'][0]['text']

'不知从什么时候开始，各家手机厂商似乎都在不约而同的做一件事情，就是把你的手机和相机的边界变得非常的模糊。很多时候你会怀疑我花钱买的这个东西到底是一台更像手机的相机，还是一台更像相机的手机。整个问题可能没有答案，但是你会看到各家厂商有各种魔改的方案，越来越离谱。那其中印象很深的就是有一台可以装上莱卡镜头的，小米公司推出的12S凹槽的概念机。当时我们说过，这应该是后无来者的一个设计的对吧？但是在前段时间小米和我们说他们新开发了一款高度保密的概念产品，它的本体是一颗小米自研的镜头，但是这个镜头里面塞下了一款M43大小的传感器，你没有听错，但是这都不是最厉害的。它最厉害的点在于你只要像这样把手机对过来，吸上去咔嚓，你就拥有了一台M43画幅等效35毫米F1.4光圈，能拍4K视频，一亿像素还具有自动对焦的相机就在你的手里。这我们没有开玩笑，这一切都是小米公司真的做的东西。所以这期节目我们一起来看一看这台也许会改变未来手机摄影方向的概念产品。你想想假如没有这个摄像头，但是给你这么一个东西，让你个随时装上去，你会想买它吗？首先我们想再强调一下，即便这一次我们是用一个评测的角度来看待它，会讲它的缺点。但是这个模块说到底是一个没有发布的概念屏，所以它有些缺陷我觉得是可以接受的。我相信如果小米真的卖它的话，应该会做的更好一点。这个玩意儿小米把它叫做模块光学系统，虽然听上去很抽象，但是它看上去就和一颗盖上后盖的镜头没有什么区别，大小也和常见的饼干镜头差不多，重量大概在100克左右，还是挺轻便的那在镜头底部有一对电子触点，还有圈磁吸，可以和手机背面这边的凹槽相连接，来进行对应的供电。当你打开原相机，然后点一点这个外部相机的图标，你会明显的感受到它会震动一下，这味着它开始工作了。但是你每次装上去都需要点一下这个图标，而且会有个明显能够感受到，等待时间会稍微有一点麻烦。而且因为这个触点真的太小了，你很难第一次就对准它把它装上去。所以往往你会需要像这样来装这个小米相机，你就得这样。就是我觉得这未必是一个缺点，因为它本质上有一点像给一个相机装镜头的感觉，虽然每次的行程是不一样的。其实像这样的模块化设计以前不是没有过。早在十年前你应该就有见过这么一个索尼叫做Q X一的相机。它当时就是一个可以装一卡口的镜头，它本身也可以成像了，但是它没有屏幕，连取景器都没有。如果你想要看到画面，那你必须通过WiFi连

In [60]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=api,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="deepseek-v3", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': '你是一个俳句诗人，擅长创作俳句，必须严格遵守俳句的格式：第一句为5个字，第二句为7个字，第三句为5个字，句子之间插入换行符。我将会给你一系列从音视频里提取的文本，请根据这些文本创作一首3行的俳句，作为对这个音视频的评价。这个评价要幽默、让人忍俊不禁。请直接输出俳句，其他内容不需要。'},
        {'role': 'user', 'content': transcription_result['transcripts'][0]['text']}],
    temperature=1.2,
    )
    

In [61]:
print(completion.choices[0].message.content)


手机变相机  
镜头底下吸光亮  
咔嚓一亿藏
